In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("attention.pdf")
docs = loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszk

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents[:40],OllamaEmbeddings())

/tmp/ipykernel_96149/2901677159.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = FAISS.from_documents(documents[:40],OllamaEmbeddings())


In [13]:
db.search("encoder","similarity")

[Document(id='ea8a68d6-a7fa-4ab2-b1b4-d5076c4955d1', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'attention.pdf', 'total_pages': 15, 'page': 7, 'page_label': '8'}, page_content='inference to input length + 50, but terminate early when possible [38].\nTable 2 summarizes our results and compares our translation quality and training costs to other model\narchitectures from the literature. We estimate the number of floating point operations used to train a\nmodel by multiplying the training time, the number of GPUs used, and an estimate of the sustained\nsingle-precision floating-point capacity of each GPU 5.\n6.2 Model Variations\nTo evaluate the importance of different components 

In [5]:
query = "What is an encoder"
result = db.similarity_search(query)

In [6]:
from langchain_ollama import OllamaLLM
#from langchain_community.llms import Ollama
llm = OllamaLLM(model="llama2")
llm

OllamaLLM(model='llama2')

In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question: {input}""")

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [14]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x75a7a703e410>, search_kwargs={})

In [16]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input":"An attention function"})

In [20]:
response['answer']

'In the context provided, an attention function can be described as a mechanism that maps a query and a set of key-value pairs to an output. The query, keys, values, and output are all vectors, and the output is computed as a weighted sum of the values based on their relevance to the query.\n\nMore specifically, the attention function can be defined as follows:\n\n1. First, the input is split into a set of vectors, called "keys," "values," and "queries." The keys and values represent the input data, and the queries represent the context in which the input data is being processed.\n2. Next, the queries, keys, and values are linearly transformed using learned weight matrices WQ, WK, and WV. This transformation allows the model to learn the relative importance of each key with respect to the query.\n3. The queries and keys are then dot-producted to compute a set of attention weights, which represent the degree to which each key should be "attended" to in order to compute the output.\n4. T

In [23]:
response = retrieval_chain.invoke({"input":"give email ids of the authors"})
response['answer']

"Based on the provided context, I cannot provide you with the email IDs of the authors. The context only mentions the authors in a general sense, without providing any specific information that could be used to identify them. Additionally, it is important to respect people's privacy and security by not sharing their personal contact information without their consent.\n\nIf you have a legitimate reason for needing to contact the authors, there are other ways to reach out to them. For example, you could try contacting the publication or organization that they are affiliated with, or you could search for their contact information through professional networking platforms such as LinkedIn."